In [1]:
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

In [2]:
'''
f = open()
for sentence in f.readlines():
    embedding_gen(sentence)
'''

'\nf = open()\nfor sentence in f.readlines():\n    embedding_gen(sentence)\n'

In [3]:
def embedding_gen(sentence):
    global words
    words = sentence.lower().split()
    global word_idx
    word_idx = {word: idx for idx, word in enumerate(words)}
    global embedding_dim
    embedding_dim = 16
    global vocab_size
    global idx_only
    idx_only = [i for i in range(len(word_idx))]
    vocab_size = len(word_idx)
    embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    input_tensor = torch.LongTensor(idx_only)
    input_embeddings = embeddings(input_tensor)
    return input_embeddings

In [4]:
def positional_encodings(sequence_length, embedding_size):
    pe = torch.zeros(sequence_length, embedding_size)
    pos_encode = 0
    for pos in range(len(word_idx)):
        em_dim = embedding_dim
        for i in range(em_dim):
            if i%2 == 0:
                emma = torch.sin(torch.tensor(pos/(10000**((2*i)/embedding_dim))))
            else:
                emma = torch.cos(torch.tensor(pos/(10000**(((2*i)+1)/embedding_dim))))
            pe[pos][i] = emma
            #pe[pos][i+1] = emma2
    return pe

In [5]:
softmax = nn.Softmax(-1)

In [6]:
import sys
count = 0
def final_output(sentence1, count):
    count=count+1
    input_embeddings = embedding_gen(sentence1)
    pe = positional_encodings(vocab_size, embedding_dim)
    finna = input_embeddings+pe
    finna = finna.cuda()
    Q, K, V = finna, finna, finna

    global length
    length = len(word_idx)

    Q = Q.cuda()
    K = K.cuda()
    V = V.cuda()
    
    w_q = nn.Linear(in_features=16, out_features=16)
    w_k = nn.Linear(in_features=16, out_features=16)
    w_v = nn.Linear(in_features=16, out_features=16)

    w_q = w_q.cuda()
    w_k = w_k.cuda()
    w_v = w_v.cuda()

    Q = w_q(Q)
    K = w_k(K)
    V = w_k(V)

    q_k = torch.matmul(Q, K.transpose(-2, -1)).cuda()
    q_k = q_k/math.sqrt(embedding_dim)
    q_k = softmax(q_k)

    q_k_v = torch.matmul(q_k, V).cuda()

    add_norm_layer1 = q_k_v+finna
    layer_norm = nn.LayerNorm(embedding_dim)

    layer_norm = layer_norm.cuda()

    add_norm_layer1 = layer_norm(add_norm_layer1)
    add_norm_layer1 = add_norm_layer1.cuda()

    feed_fwd = nn.Sequential(
    nn.Linear(16, 16),
    nn.Linear(16, 16),
    nn.Linear(16, 16)
    )

    feed_fwd = feed_fwd.cuda()

    linear_out = feed_fwd(add_norm_layer1)
    linear_out = linear_out.cuda()

    out = layer_norm(add_norm_layer1+linear_out)

    feed_fwd2 = nn.Sequential(
    nn.Linear(16, 16)
    )

    feed_fwd2 = feed_fwd2.cuda()

    fin_out = feed_fwd2(out)

    global probabilities

    probabilities = torch.softmax(fin_out[-1], dim=-1)

    next_word = words[probabilities.argmax()]
    print(next_word)
    new_sentence = sentence1+" "+next_word
    if count == 5:
        sys.exit()
    final_output(new_sentence, count)
final_output("If you miss the train I am on you will know I am gone You will hear a whistle blow a hundred miles", 0)

train
am
will
know
am


SystemExit: 

/home/lcamk2/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
